In [1]:
using Random
using MatrixFreeNewton

## [n-dimensional Rosenbrock](https://arxiv.org/pdf/1903.09556.pdf)


$\sum_{i=1}^{n-1} [100(x_{i+1} - x_i^2)^2 + (1-x_i)^2]$

In [2]:
function rosenbrockn(x)
    f = 0.0
    for i = 1:size(x)[1]-1
        f += 100*(x[i+1] - x[i]^2)^2 + (1 - x[i])^2
    end
    return f
end

n = 10
print("Now for gradient descent \n")
w_star_gd,logger_gd = gradientDescent(rosenbrockn,zeros(n))

print("Now for curvature gradient descent \n")
w_star_gd,logger_curve_gd = curvatureScaledGradientDescent(rosenbrockn,zeros(n))

print("Now for Newton \n")
w_star_newton,logger_full_newton = fullNewton(rosenbrockn,zeros(n),printing_frequency = 1,
                                                                alpha = 1e0,iterations=2*n)



Now for gradient descent 
At initial guess obj = 9.0
At iteration 10 obj = NaN
At iteration 20 obj = NaN
At iteration 30 obj = NaN
At iteration 40 obj = NaN
At iteration 50 obj = NaN
At iteration 60 obj = NaN
At iteration 70 obj = NaN
At iteration 80 obj = NaN
At iteration 90 obj = NaN
At iteration 100 obj = NaN
Now for curvature gradient descent 
At initial guess obj = 9.0
At iteration 10 obj = 8.709303623211357
At iteration 20 obj = 8.487770557748984
At iteration 30 obj = 8.28198689755699
At iteration 40 obj = 8.098128549963612
At iteration 50 obj = 7.933955800888958
At iteration 60 obj = 7.785872991823892
At iteration 70 obj = 7.650753325311406
At iteration 80 obj = 7.52615060336678
At iteration 90 obj = 7.410188188400873
At iteration 100 obj = 7.301422591306192
Now for Newton 
At initial guess obj = 9.0
At iteration 1 obj = 105.93924292104158
At iteration 2 obj = 126.6205786600929
At iteration 3 obj = 182.85400113825946
At iteration 4 obj = 343.6333994179984
At iteration 5 obj = 43

([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], MatrixFreeNewton.NewtonLogger(1.0, [9.0, 105.93924292104158, 126.6205786600929, 182.85400113825946, 343.6333994179984, 436.3586171045547, 252.56686372653868, 526.0980684641479, 416.6633226446785, 0.7606102988509199  …  0.11005306862251926, 0.7667041586551342, 4.274421151304933e-6, 1.207954020517505e-11, 8.435625581350841e-21, 0.0, 0.0, 0.0, 0.0, 0.0]))

In [3]:
print("Now for low rank SFNewton with full Hessian \n")
w_star_newton,logger_full_sfn = lowRankSaddleFreeNewton(rosenbrockn,zeros(n),
                                                            iterations = 2*n,rank = n)

half_rank = Int(floor(0.9*n))
print("Now for low rank SFNewton with reduced Hessian with half rank \n")
println("half rank = ",half_rank)
w_star_newton,logger_lrsfn = lowRankSaddleFreeNewton(rosenbrockn,zeros(n),alpha = 1e-4,
                                                    gamma = 1e-2,iterations = 2*n,rank = half_rank)

# print("Now for low rank SFNewton with gradient based Hessian \n")
# w_star_newton,trace_newton = lowRankSaddleFreeNewton(rosenbrockn,zeros(n),hessian = "from_gradient",
#                                                             iterations = 2*n,history = false,rank = n)

Now for low rank SFNewton with full Hessian 
At initial guess obj = 9.0

At iteration 10 obj = 3.8105973341977375 lambda_1 = 3007.775619391618 lambda_r = 0.2993543154612438
At iteration 20 obj = 0.0 lambda_1 = 1762.8408814945562 lambda_r = 0.4987517495768474
Now for low rank SFNewton with reduced Hessian with half rank 
half rank = 9
At initial guess obj = 9.0

At iteration 10 obj = 8.555424352834542 lambda_1 = 235.28980103640703 lambda_r = 197.32720951608903
At iteration 20 obj = 8.075658936963944 lambda_1 = 342.7731327919445 lambda_r = 189.58015944578548


([0.39607416510950777, 0.15080531908710137, 0.018670881998069818, 0.0035513570910272897, -0.002253241075179418, 0.009274092597504117, -0.007950681730175472, -0.00014529326395896995, -0.0024691976744062984, -0.006389371030756562], MatrixFreeNewton.SFNLogger(0.0001, [9.0, 8.960276216274545, 8.919192368721983, 8.876848393122758, 8.833358222597024, 8.788845645129934, 8.743439783926007, 8.697270694566306, 8.650465526789011, 8.603145572425682  …  8.507406732776987, 8.459188917000901, 8.410859107121233, 8.36249856914243, 8.314182874799288, 8.265983117791643, 8.217966955434393, 8.170199377559651, 8.122743152001071, 8.075658936963944], [202.0000000000002 202.35845069865906 … 329.6751060739241 342.7731327919445; 202.00000000000006 202.15809678114442 … 207.41072587000286 207.81216914220315; … ; 201.87747038113054 201.84206921990733 … 198.2527899081301 197.99936278014235; 199.97303379668762 199.97178167168573 … 190.18304112387693 189.58015944578548]))

In [4]:
logger_gd.losses

101-element Vector{Float64}:
    9.0
 4809.0
    8.255681876505609e15
    5.951969133663163e53
    5.373886071433855e167
   Inf
   Inf
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
    ⋮
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN
  NaN

In [5]:
logger_lrsfn.losses

21-element Vector{Float64}:
 9.0
 8.960276216274545
 8.919192368721983
 8.876848393122758
 8.833358222597024
 8.788845645129934
 8.743439783926007
 8.697270694566306
 8.650465526789011
 8.603145572425682
 8.555424352834542
 8.507406732776987
 8.459188917000901
 8.410859107121233
 8.36249856914243
 8.314182874799288
 8.265983117791643
 8.217966955434393
 8.170199377559651
 8.122743152001071
 8.075658936963944

In [6]:
logger_lrsfn.spectra[:,1]

9-element ElasticArrays.ElasticVector{Float64, Vector{Float64}}:
 202.0000000000002
 202.00000000000006
 202.0
 201.99999999999997
 201.99999999999997
 201.99999999999994
 201.9999999999999
 201.87747038113054
 199.97303379668762